# Oanda Demo Trading Notebook

## Packages

Normal Packages

In [1]:
import numpy as np
import pandas as pd

import yaml
import json

import time
import datetime
import winsound
import collections

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

Oanda Packages

In [2]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts

import oandapyV20.definitions.pricing as defpricing
import oandapyV20.endpoints.instruments as instruments

Custom packages

In [3]:
from utils.toolsapi import get_date_time
from utils.toolsapi import get_prices
from utils.toolsapi import calc_duration

## Functions

In [4]:
def terminate_connection():
    try:
        r.terminate(message = "maxrecs records received")
    except:
        pass

In [5]:
def get_targets(start_price, target_num = 3):
    positive_targets = {}
    negative_targets = {}
    pip = 10**-4
    
    for i in range(target_num):
        move_val = pip*(i+1)
        positive_targets[i] = start_price + move_val
        negative_targets[i] = start_price - move_val
        
    return(positive_targets, negative_targets)

## API Setup

Read from config file

In [6]:
config_file = 'config/access_token.yaml'

with open(config_file) as c_file:
    config = yaml.load(c_file)

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']

api = API(access_token = access_token)

## Code Engine

### Real Live data streamer

In [7]:
params = {
    'instruments': 'EUR_USD'
}

num = 600
first_run_flag = 0
target_num = 3

reached_targets = {'start_price' : 0,
                   'target_num' : 0,
                   'positive_val' : 0,
                   'negative_val' : 0,
                   'positive' : 0,
                   'p_duration' : '',
                   'n_duration' : '',
                   'negative' : 0}

df_reached_targets = pd.DataFrame()
tick_list = []
first_target_flag = 'not_reached'

In [8]:
r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)

start_time = time.time()

for i, resp in tqdm(enumerate(rv)):
    
    if i < num: # Check if we are within the required number of price iterations               
        resp_type = resp['type']       
        
        if resp_type == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
            pass
            #print(resp_type)
        
        elif resp_type == 'PRICE': # Check whether it is a price response                 
            date_val, time_val, time_fraction = get_date_time(resp) # Get time stamp for reference            
            sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response                      
            tick_list.append(tick_price)
            
            if first_run_flag == 0:
                positive_targets, negative_targets = get_targets(tick_price, target_num)
                first_run_flag = 1
                for j in range(target_num):
                    df_reached_targets = df_reached_targets.append(reached_targets, ignore_index = True)
                    df_reached_targets.loc[df_reached_targets.index[j], 'target_num'] = j+1
                    df_reached_targets.loc[df_reached_targets.index[j], 'start_price'] = tick_price
                    df_reached_targets.loc[df_reached_targets.index[j], 'positive_val'] = positive_targets[j]
                    df_reached_targets.loc[df_reached_targets.index[j], 'negative_val'] = negative_targets[j]

            for k in range(target_num):
                if tick_price >= positive_targets[k]:
                    df_reached_targets.loc[df_reached_targets.index[k], 'positive'] += 1
                    if first_target_flag == 'not_reached':
                        end_time = time.time()    
                        duration = calc_duration(start_time, end_time)
                        df_reached_targets.loc[df_reached_targets.index[k], 'p_duration'] = duration
                        first_target_flag == 'reached'
                        
                if tick_price <= negative_targets[k]:
                    df_reached_targets.loc[df_reached_targets.index[k], 'negative'] += 1            
                    if first_target_flag == 'not_reached':
                        end_time = time.time()    
                        duration = calc_duration(start_time, end_time)
                        df_reached_targets.loc[df_reached_targets.index[k], 'n_duration'] = duration
                        first_target_flag == 'reached'                        
                        
    else: # Crossed the required number of price iterations
        terminate_connection()


winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

601it [05:54,  1.69it/s]


In [9]:
print(f'positive_targets : {positive_targets}')
print(f'negative_targets : {negative_targets}')
df_reached_targets =  df_reached_targets[['start_price', 'target_num', 'positive_val', 'negative_val','positive', 'negative','p_duration','n_duration']]
df_reached_targets

positive_targets : {0: 1.13794, 1: 1.13804, 2: 1.13814}
negative_targets : {0: 1.13774, 1: 1.13764, 2: 1.13754}


,start_price,target_num,positive_val,negative_val,positive,negative,p_duration,n_duration
0,1.13784,1.0,1.13794,1.13774,473.0,0.0,0:5:50,
1,1.13784,2.0,1.13804,1.13764,409.0,0.0,0:5:50,
2,1.13784,3.0,1.13814,1.13754,315.0,0.0,0:5:37,
